# process_bike_counts

In [3]:
# imports
import pandas as pd
import geopandas as gpd
import numpy as np

## Step 1 - Process Bike Counts Data
The following code processes counts of bicycle traffic taken from this [link](https://docs.google.com/spreadsheets/d/1urP-ZA0Pd25_JZZ18hkGPlDEUQusBp49XmLzwpZ-2ag/edit#gid=864700651). Goal is to get the counts for each location as a row in a dataframe, then aggregate by sector (NW, SW, SE...).

In [4]:
FILE = 'data/Portland Bike Counts.csv'

df = pd.read_csv(FILE, header=1)

In [6]:
# Melt the dataframe so each row is a reading
df = df.melt(['Sector','Site #','Location'])
df

,Sector,Site #,Location,variable,value
0,bridge,2,Burnside Bridge,2018,"1,780"
1,bridge,4,Sellwood Bridge,2018,735
2,north,6,N Fessenden & Peninsula Crossing,2018,220
3,north,7,N Interstate & Greeley,2018,"1,420"
4,north,8,N Interstate & Larrabee,2018,"1,660"
5,north,9,N Portsmouth & Houghton,2018,200
6,north,10,N Denver & Rosa Parks,2018,850
7,north,11,N Vancouver & Russell,2018,"4,705"
8,north,12,N Williams & Russell,2018,"4,108"
9,north,13,N Smith & St. Louis,2018,20


In [7]:
# Filter out NaNs and '-'s
df['value'] = pd.to_numeric(df['value'], errors='coerce', downcast='integer')
df = df[pd.notnull(df['value'])]

In [62]:
# Group by the sector
df_agg = df.groupby(['Sector','variable'])['value'].sum()
df_agg = df_agg.reset_index()
df_agg = df_agg.set_index('Sector')

In [63]:
# Collapse all of the years into a JSON object for each sector
df_agg = df_agg.groupby('Sector').apply(lambda x: x.set_index('variable').to_json(orient='columns'))

In [65]:
df_agg

Sector
bridge         {"value":{"2001":965.0,"2003":690.0,"2006":500...
city center    {"value":{"2000":1140.0,"2001":1590.0,"2002":9...
east           {"value":{"2006":1115.0,"2007":2155.0,"2008":4...
north          {"value":{"2000":2515.0,"2001":2825.0,"2003":3...
northeast      {"value":{"2000":2130.0,"2001":2565.0,"2003":2...
northwest      {"value":{"2000":710.0,"2001":255.0,"2002":445...
southeast      {"value":{"2000":1375.0,"2001":3105.0,"2003":1...
southwest      {"value":{"2000":490.0,"2001":485.0,"2003":720...
dtype: object

# Step 2 - Generate GEOJSON File

This code uses geopandas to generate a GEOJSON object using a shape file for Portland's boundaries, joined with the above dataset. Geopandas was difficult to install on Windows but I found [this](https://geoffboeing.com/2014/09/using-geopandas-windows/) link very helpful. Shape files downloaded from [here](https://gis-pdx.opendata.arcgis.com/datasets/portland-administrative-sextants).

In [66]:
FILE = 'data/Portland_Administrative_Sextants.shp'
OUT_FILE = 'vis/pdx_zones.geojson'

# Read the shape file with Portland boundaries
shp = gpd.read_file(FILE)

In [67]:
shp

,OBJECTID,FID_quadra,PREFIX,acres,sqmiles,Quadrants,FID_South_,Shape_Leng,Shape_Area,geometry
0,1,2,NW,14532.835938,22.707556,NorthWest,-1,68370.581202,1.628050e+08,POLYGON ((-122.7925059896695 45.61975437764568...
1,2,5,SE,28707.505859,44.855480,SouthEast,-1,66324.738076,2.396703e+08,POLYGON ((-122.6519986985515 45.52289597099575...
2,3,4,SW,14469.660156,22.608845,South,1,33350.409679,2.290833e+07,POLYGON ((-122.6703018158715 45.48586233144766...
3,4,4,SW,14469.660156,22.608845,SouthWest,-1,54492.289907,1.062193e+08,POLYGON ((-122.6673597901969 45.52305133547165...
4,5,4,SW,14469.660156,22.608845,SouthWest,-1,1863.970732,2.315196e+04,POLYGON ((-122.6658845867188 45.50341050137209...
5,6,3,N,26102.648438,40.785385,North,-1,55159.484521,1.570435e+08,POLYGON ((-122.6668008350709 45.52797063956169...
6,7,3,NE,26102.648438,40.785385,NorthEast,-1,75437.454095,2.153998e+08,"POLYGON ((-122.546027143023 45.57725776175279,..."


In [68]:
shp['Quadrants'] = shp['Quadrants'].str.lower()

In [75]:
#shp
shp2 = shp.merge(df_agg.rename('Bike_Counts'), left_on='Quadrants', right_on='Sector', how='inner')

In [76]:
# Convert the shape file to GEOJSON format
shp2.to_file(OUT_FILE, driver='GeoJSON')